In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [1]:
from collections import defaultdict
import pickle
import torch
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
import torchvision

In [3]:
with open("/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/ReLUs/ReLUs_train.pkl", 'rb') as iffile:
    features = pickle.load(iffile)

In [4]:
layer = 'flatten'
features = features[layer]
features_matrix = None

for i in range(len(features)):
    if isinstance(features_matrix, torch.Tensor):
        features_matrix = torch.cat((features_matrix, features[i]))
    else:
        features_matrix = features[i]

features = None

In [5]:
features_matrix = torch.flatten(features_matrix, start_dim=1, end_dim=-1)

In [6]:
# Load adversarial samples
with open("/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/ReLUs/ReLUs_test_adversarial.pkl", 'rb') as iffile:
    adv_features = pickle.load(iffile)

layer = 'flatten'
adv_features = adv_features[layer]
adv_features_matrix = None

for i in range(len(adv_features)):
    if isinstance(adv_features_matrix, torch.Tensor):
        adv_features_matrix = torch.cat((adv_features_matrix, adv_features[i]))
    else:
        adv_features_matrix = adv_features[i]

adv_features = None

In [18]:
X_ben = features_matrix[0:512, :]
y_ben = np.ones((512))

#X_test = features_matrix[:15000, :]
X_adv = adv_features_matrix
y_adv= -np.ones((512))

In [17]:
X_train = np.concatenate((X_ben, X_adv), axis=0)

In [19]:
y_train = np.concatenate((y_ben, y_adv), axis=0)

In [20]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [21]:
X_train, y_train = unison_shuffled_copies(X_train, y_train)

In [93]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC

clf = SVC(C=0.7, gamma=0.075)
clf.fit(X_train, y_train)

SVC(C=0.7, gamma=0.075)

In [94]:
res_adv = clf.predict(X_adv)

In [95]:
len(np.where(res_adv == -1)[0]) / 512

0.90625

In [96]:
res_ben = clf.predict(X_ben)

In [97]:
len(np.where(res_ben == 1)[0]) / 512

0.927734375

In [1]:
import pickle

In [2]:
with open("/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/RBF/RBF_CW_5120.pkl", 'rb') as iff:
    clf = pickle.load(iff)

In [3]:
clf

SVC(C=0.7, gamma=0.075)

In [27]:
# Quantize
# Comput per-dimension mean

In [ ]:
dim_mean = features_matrix.mean(dim=0)

In [ ]:
binarized_matrix = torch.where(features_matrix >= dim_mean, torch.ones_like(features_matrix), torch.zeros_like(features_matrix))

In [ ]:
features_matrix = None

In [ ]:

torch.cuda.empty_cache()

In [ ]:
binarized_matrix = binarized_matrix.cpu().numpy()

In [ ]:


model = OneClassSVM(kernel='rbf', gamma=0.0000000001)
model.fit(features_matrix)

In [1]:
# Filtering out stuff

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from utils import configs
import hashlib
sys.path.append("/home/zsarwar/Projects/SparseDNNs/adversarial-attacks-pytorch")
import torchattacks

from utils.utils_2 import imshow, get_pred
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import DataLoader, Subset
import numpy as np
from utils.rbf_model import RBF_SVM

In [2]:
import torch.nn as nn
import torch
import pickle


class RBF_SVM(nn.Module):
    def __init__(self, clf):
        super().__init__()
        self.SV = nn.Parameter(torch.from_numpy(clf.support_vectors_))
        self.IC = nn.Parameter(torch.tensor(clf.intercept_))
        self.DC = nn.Parameter(torch.from_numpy(clf.dual_coef_))
        self.gamma = nn.Parameter(torch.tensor(clf.gamma))
        self.SV.requires_grad = True
        self.IC.requires_grad = True
        self.DC.requires_grad = True
        self.gamma.requires_grad = True

    def forward(self, X):
        X_expanded = X.unsqueeze(1)
        SV_broadcasted = torch.broadcast_to(self.SV, (X.shape[0], self.SV.shape[0], self.SV.shape[1]))
        res_batch = torch.subtract(SV_broadcasted, X_expanded)
        res_batch = torch.norm(res_batch, p=2, dim=2)
        res_batch = torch.square(res_batch)
        res_batch = torch.multiply(-self.gamma, res_batch)
        res_batch = torch.exp(res_batch)
        res_batch = torch.multiply(res_batch, self.DC)
        res_batch = torch.sum(res_batch, dim=1)
        res_batch = res_batch + self.IC
        return res_batch


In [5]:
rbf_path = "/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/RBF/RBF_CW_5120.pkl"
with open(rbf_path,'rb') as in_model:
    clf = pickle.load(in_model)
    
# Class RBF model
rbf_svm = RBF_SVM(clf)

In [3]:
# GO-GO-GO!
normalize  =  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    

# Undo normalization for now
transformation = transforms.Compose([
    transforms.ToTensor(), 
    ])

In [5]:
trainset = torchvision.datasets.CIFAR10(root='/bigstor/zsarwar/CIFAR10', train=True,
                                            download=False, transform=transformation)
random_indices = np.random.randint(low=0, high = len(trainset), size=(512))
trainset = Subset(trainset, indices=random_indices)
dataloader = DataLoader(trainset, batch_size=32,
                                            shuffle=True, num_workers=2)

In [2]:
model = resnet18(
    
)

In [3]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  